In [1]:
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

In [2]:
design = designs.DesignPlanar()

In [3]:
design.overwrite_enabled = True
design.chips.main
design.chips.main.size.size_x = '4cm'
design.chips.main.size.size_y = '5cm'
gui = MetalGUI(design)

In [4]:
design.variables['cpw_width'] = '.1mm'
design.variables['cpw_gap'] = '.1mm'

## LAYOUT
---

In [5]:
from qiskit_metal.qlibrary.tlines import meandered, straight_path, framed_path
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.couplers.line_tee import LineTee

In [6]:
"""
Transmission Line In/Out Ports 
"""
port_1_options = Dict(pos_x='0mm',pos_y='0mm',orientation='0')
port_1_termination = OpenToGround(design,'Port1_Termination',options=port_1_options)

port_2_options = Dict(pos_x='5cm',pos_y='0mm',orientation='180')
port_2_termination = OpenToGround(design,'Port2_Termination',options=port_2_options)

In [7]:
"""
Transmission Line Tee
"""
tee1_options = Dict(pos_x="2.5cm",pos_y="0cm",rotation='0',coupling_length='2.5mm',down_length='1mm',
                    fillet='.04cm',prime_gap='100um',
                    prime_width=design.variables['cpw_width'], second_width=design.variables['cpw_width'])
tee1 = CoupledLineTee(design, "CouplingTee",options=tee1_options)

In [8]:
"""
Transmission Line Thru
"""
straighta_tline_options = Dict( pin_inputs = Dict(  start_pin=Dict(component='Port1_Termination', pin='open'),
                                                    end_pin=Dict(component='CouplingTee', pin='prime_start')))
straighta_tline = straight_path.RouteStraight(design,'straighta_tline', options=straighta_tline_options)

straightb_tline_options = Dict( pin_inputs = Dict(  start_pin=Dict(component='CouplingTee', pin='prime_end'),
                                                    end_pin=Dict(component='Port2_Termination', pin='open')))
straightb_tline = straight_path.RouteStraight(design,'straightb_tline', options=straightb_tline_options)

In [9]:
"""
Resonator Grounding
"""
res_short_options = Dict(pos_x='2.62cm',pos_y='-.75cm',orientation='-90')
res_short = ShortToGround(design,'Res_short',options=res_short_options)
"""
Resonator A
"""
meanderA_options = Dict(
    pin_inputs = Dict(  start_pin=Dict(component='CouplingTee', pin='second_end'),
                        end_pin=Dict(component='Res_short', pin='short')), 
                        lead = Dict(start_straight='800um',end_straight='10um'),
                        total_length = '2.1cm', fillet = '50um', width=design.variables['cpw_width'])
MeanderA = meandered.RouteMeander(design,'meanderA', options=meanderA_options)

In [ ]:
"""
DC bias Tee
"""
dc_tee_options = Dict(  pos_x='2.625cm', pos_y='-1.mm',mirror=False,t_length='200um',
                        prime_width=design.variables['cpw_width'], orientation='-90', 
                        second_width=design.variables['cpw_width'] )
dc_tee=LineTee(design, "DcTee",options=dc_tee_options)
"""
DC Bias Pad
"""
port_3_options = Dict(pos_x='0mm',pos_y='-1cm',orientation='0') # control
port_3_termination = LaunchpadWirebondCoupled(design,'Port3_Termination',options=port_3_options)
"""
DC Bias Line
"""
control_line_options = Dict( pin_inputs=Dict(   start_pin=Dict(component='DcTee', pin='second_end'),
                                                end_pin=Dict(component='Port3_Termination', pin='tie')),
                                                total_length = '1mm', fillet = '99.00um',
                                                width=design.variables['cpw_width'],)
ControlLine=framed_path.RouteFramed(design,'controlline',options=control_line_options)

In [10]:
gui.rebuild()
gui.zoom_on_components([tee1.name, #resonatorA.name,
                        port_1_termination.name, port_2_termination.name])

In [ ]:
gui.figure.savefig('tline.png')
from IPython.display import Image, display
_disp_ops = dict(width=500)
display(Image('tline.png', **_disp_ops))

# Analysis

In [ ]:
from qiskit_metal.analyses.simulation import ScatteringImpedanceSim
em1 = ScatteringImpedanceSim(design, "hfss")